<a href="https://colab.research.google.com/github/DmitrijLeontev/Dmitrij_Leontev/blob/main/%D0%B4%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5%20%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%94%D0%97_Pro_%7C_5_%D0%B7%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D0%B5_%7C_ChatGPT_Professional_%7C_%D0%A3%D0%98%D0%98_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

На основании нормативов по качеству питьевой воды и прайс листа с ценами бутилированной воды создайте нейро-консультанта, который бы отвечал на вопросы по данной информации.
Для этого скачайте pdf файлы по ссылкам:

**Нормативы**: https://files.stroyinf.ru/Data2/1/4294850/4294850590.pdf

https://files.stroyinf.ru/Data2/1/4294846/4294846957.pdf

**Прайс-лист**: https://xn---56-5cdablc8cp7g.xn--p1ai/ckfinder/userfiles/files/price%20ot%202019-01-01.pdf

Документы с нормативами объедините в 1 БЗ, а из прайс-листа сделайте вторую БЗ.

Нейро-консультант должен на основании вопроса пользователя динамически выбирать соответствующую базу знаний и отвечать на оcнове нее на вопрос.

Проверьте работу нейро-консультанта на самостоятельно-сформулированных вопросах.

In [ ]:
!pip install unstructured pdf2image pdfminer.six faiss-cpu==1.7.4

In [ ]:
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
# функция принимает URL PDF-файла и путь, по которому файл будет сохранен в файловой системе Colab.
# Она скачивает файл и сохраняет его, если HTTP-запрос успешен
def download_pdf_from_url(url, save_path):
    response = requests.get(url)

    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"PDF downloaded and saved as {save_path}")
    else:
        print(f"Failed to download PDF. HTTP Status Code: {response.status_code}")

# используем функцию выше
url =  "https://files.stroyinf.ru/Data2/1/4294850/4294850590.pdf"
save_path = "/content/4294850590.pdf"

download_pdf_from_url(url, save_path)

PDF downloaded and saved as /content/4294850590.pdf


In [ ]:
url = "https://files.stroyinf.ru/Data2/1/4294846/4294846957.pdf"
save_path = "/content/4294846957.pdf"

download_pdf_from_url(url, save_path)

PDF downloaded and saved as /content/4294846957.pdf


In [ ]:
# функция, которая принимает список названий PDF-файлов и возвращает базу знаний FAISS
def create_faiss_vectorstore(pdf_names):
    pdf_folder_path = "/content/"

    # Фильтрация файлов: загрузка только файлов с определенными именами и расширением .pdf
    all_files = os.listdir(pdf_folder_path)
    pdf_files = [fn for fn in all_files if fn in pdf_names and fn.endswith('.pdf')]

    # Создание объектов UnstructuredPDFLoader для каждого PDF-файла
    loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in pdf_files]

    all_documents = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100, length_function=len)

    for loader in loaders:
        # Загрузка и разделение текста
        raw_documents = loader.load()
        documents = text_splitter.split_documents(raw_documents)
        all_documents.extend(documents)

    # Создание FAISS vectorstore
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(all_documents, OpenAIEmbeddings()).as_retriever()

    return vectorstore

In [ ]:
#Создадим базу знаний по норматиавм питьевой воды на основе спецификаций в формате pdf:
pdf_names = ['4294850590.pdf', '4294846957.pdf']
db_standards = create_faiss_vectorstore(pdf_names)

In [ ]:
# Используем MultiRetrievalQAChain для создания цепочки вопросов-ответов, которая выбирает цепочку поиска QA (вопрос-ответ)
# Для этого создадим базу знаний еще и по прайслисту из pdf
# Сохраняем pdf в память колаба
url = "https://xn---56-5cdablc8cp7g.xn--p1ai/ckfinder/userfiles/files/price%20ot%202019-01-01.pdf"
save_path = "/content/price%20ot%202019-01-01.pdf"

download_pdf_from_url(url, save_path)

PDF downloaded and saved as /content/price%20ot%202019-01-01.pdf


In [ ]:
# создаем базу знаний faiss по prise
pdf_names = ['price%20ot%202019-01-01.pdf']
db_prise = create_faiss_vectorstore(pdf_names)

In [ ]:
# Таким образом, у нас имеются 2 разные базы знаний faiss: одна по Corolla, вторая по LandCruiser
# Добавляем информацию и описание для наших векторных хранилищ (баз знаний)
retriever_infos = [
    {
        "name": "drinking_water_quality_standards",
        "description": "Suitable for answering questions about drinking water quality standards",  # описываем, для каких вопросов применять данную БЗ
        "retriever": db_standards
    },
    {
        "name": "_price_bottled_water",
        "description": "Suitable for answering questions about the price and quantity of drinking bottled water", # описываем, для каких вопросов применять данную БЗ
        "retriever": db_prise
    }
]

In [ ]:
# Иициализируем класс
chain = MultiRetrievalQAChain.from_retrievers(ChatOpenAI(temperature=0), retriever_infos, verbose=True)

In [ ]:
# Запускаем на вопросе из раздела по нормативам по качеству питьевой воды (в промежуточных итогах видим , что выбрана нужная база данных )

print(chain.run("Каким нормативам должно соответствовать качество воды?"))



> Entering new MultiRetrievalQAChain chain...
drinking_water_quality_standards: {'query': 'Каким нормативам должно соответствовать качество воды?'}
> Finished chain.
Качество питьевой воды должно соответствовать требованиям действующих санитарных правил и норм, утвержденных в установленном порядке.


In [ ]:
# задаем вопрос, на основе вопроса выбирается необходимая БЗ
print(chain.run("Применимы ли данные нормативы при проведении сертификации воды, подаваемой централизованными системами питьевого водоснабжения?"))



> Entering new MultiRetrievalQAChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


drinking_water_quality_standards: {'query': 'Are these standards applicable when certifying water supplied by centralized drinking water supply systems?'}
> Finished chain.
Yes, these standards are applicable when certifying water supplied by centralized drinking water supply systems.


In [ ]:
# задаем вопрос, на основе вопроса выбирается необходимая БЗ
print(chain.run("Сколько стоит домтавка воды  Аква-Вита?"))



> Entering new MultiRetrievalQAChain chain...
bottled_water_price: {'query': 'Сколько стоит доставка воды Аква-Вита?'}
> Finished chain.
Доставка продукции ООО "Аква-Вита" бесплатна при заказе от 2 бутылей воды (18,9л) или от 10 упаковок.


In [ ]:
# задаем вопрос, на основе вопроса выбирается необходимая БЗ
print(chain.run("Подскажи контактный телефон для заключения договора доставки воды Аква-Вита?"))



> Entering new MultiRetrievalQAChain chain...
None: {'query': 'Подскажи контактный телефон для заключения договора доставки воды Аква-Вита?'}
> Finished chain.
К сожалению, я не могу найти контактный телефон для заключения договора доставки воды Аква-Вита.
